In [150]:
import pandas as pd

In [151]:
df = pd.read_csv('/content/sample_data/papers.csv',on_bad_lines='warn')

In [152]:
df.head()
df

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."
...,...,...,...,...,...,...,...
7236,994,1994,Single Transistor Learning Synapses,NaN,994-single-transistor-learning-synapses.pdf,Abstract Missing,Single Transistor Learning Synapses\n\nPaul Ha...
7237,996,1994,"Bias, Variance and the Combination of Least Sq...",NaN,996-bias-variance-and-the-combination-of-least...,Abstract Missing,"Bias, Variance and the Combination of\nLeast S..."
7238,997,1994,A Real Time Clustering CMOS Neural Engine,NaN,997-a-real-time-clustering-cmos-neural-engine.pdf,Abstract Missing,A Real Time Clustering CMOS\nNeural Engine\nT....
7239,998,1994,Learning direction in global motion: two class...,NaN,998-learning-direction-in-global-motion-two-cl...,Abstract Missing,Learning direction in global motion: two\nclas...


In [153]:
df.shape

(7241, 7)

In [154]:
df = df.iloc[:5000,:]
df

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."
...,...,...,...,...,...,...,...
4995,5522,2014,Low-Rank Time-Frequency Synthesis,Poster,5522-low-rank-time-frequency-synthesis.pdf,Many single-channel signal decomposition techn...,Low-Rank Time-Frequency Synthesis\n\nMatthieu ...
4996,5523,2014,A State-Space Model for Decoding Auditory Atte...,Poster,5523-a-state-space-model-for-decoding-auditory...,Humans are able to segregate auditory objects ...,A State-Space Model for Decoding Auditory\nAtt...
4997,5524,2014,Efficient Structured Matrix Rank Minimization,Poster,5524-efficient-structured-matrix-rank-minimiza...,We study the problem of finding structured low...,Efficient Structured Matrix Rank Minimization\...
4998,5525,2014,Efficient Minimax Signal Detection on Graphs,Poster,5525-efficient-minimax-signal-detection-on-gra...,"Several problems such as network intrusion, co...",Ef?cient Minimax Signal Detection on Graphs\n\...


In [155]:
df.shape

(5000, 7)

In [156]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [157]:
stop_words = set(stopwords.words('english'))

Creating a list of custom stop words

In [158]:
new_words = ["fig","figure","image","sample","using","show","result","large","also","one","two","three","four","five","seven","eight","nine"]

In [159]:
stop_words = list(stop_words.union(new_words))

In [160]:
def preprocess_text(text):
  if isinstance(text, float):
    text = str(text)
  text = text.lower()
  text = re.sub(r'<.*?>',' ',text)
  text = re.sub(r'[^a-zA-Z]',' ',text)
  text = nltk.word_tokenize(text)
  text = [word for word in text if word not in stop_words]
  text = [word for word in text if len(word) >= 3]
  stemming = PorterStemmer()
  text = [stemming.stem(word) for word in text]
  return ' '.join(text)

In [161]:
preprocess_text("tHiS iS to learning and understanding 4545 %$# PyThoN <h1><p> web language</p></h1>")

'learn understand python web languag'

In [162]:
docs = df['paper_text'].apply(lambda x: preprocess_text(x))

In [163]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer(max_df = 0.95, max_features = 5000, ngram_range = (1,3))
word_count_vectors = count_vector.fit_transform(docs)
word_count_vectors

<5000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 3183711 stored elements in Compressed Sparse Row format>

In [164]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=True,smooth_idf=True)
tfidf_transformer = tfidf_transformer.fit(word_count_vectors)
tfidf_transformer

TfidfTransformer()

In [165]:
feature_names = count_vector.get_feature_names_out()
for feature in feature_names:
  print(feature)

aaai
ab
abil
abl
absenc
absolut
absolut valu
abstract paper
abstract present
acad
acad sci
academ
academ press
academi
academi scienc
acceler
accept
access
accommod
accomplish
accord
accordingli
account
accumul
accur
accur estim
accuraci
achiev
acid
acknowledg
acknowledg author
acknowledg research
acknowledg support
acknowledg thank
acknowledg work
acknowledg work support
acl
acm
acoust
acoust speech
acoust speech signal
acquir
acquisit
across
across differ
act
action
action pair
action potenti
action select
action space
action state
action valu
action valu function
activ
activ function
activ learn
activ neuron
activ pattern
activ set
actor
actor critic
actual
acycl
ad
adaboost
adam
adapt
add
addit
addit model
addit nois
address
address issu
address problem
adequ
adjac
adjac matrix
adjust
admiss
admit
admm
adopt
adult
advanc
advanc neural
advanc neural inform
advantag
adversari
advertis
advic
affect
affer
affin
agarw
age
agent
aggreg
aggress
agnost
agre
agreement
ahead
aid
aij
aim
aist

In [166]:
def get_keywords(idx,docs,topN=10):
  docs_word_count = tfidf_transformer.transform(count_vector.transform([docs[idx]]))
  #build sparse matrix
  docs_word_count = docs_word_count.tocoo()
  tuples = zip(docs_word_count.col,docs_word_count.data)
  sorted_items = sorted(tuples,key = lambda x: (x[1],x[0]),reverse=True)

  #getting the top N keywords
  sorted_items = sorted_items[:topN]
  score_vals = []
  feature_vals = []
  for idx, score in sorted_items:
    score_vals.append(round(score,3))
    feature_vals.append(feature_names[idx])

  #we get the final result here
  results = {}
  for idx in range(len(feature_vals)):
    results[feature_vals[idx]] = score_vals[idx]
  return results

def print_keywords(idx,keywords,idf):
  print("\n--------Title-------")
  print(df['title'][idx])
  print("\n--------Abstract-------")
  print(df['abstract'][idx])
  print("\n--------Keywords-------")
  for k in keywords:
    print(k,keywords[k])


In [177]:
#in order to get the indices for all the rows with non-empty paper text info
df['paper_text'][4550]

'Multilinear Dynamical Systems\nfor Tensor Time Series\nMark Rogers\nLei Li\nStuart Russell\nEECS Department, University of California, Berkeley\nmarkrogersjr@berkeley.edu, {leili,russell}@cs.berkeley.edu\n\nAbstract\nData in the sciences frequently occur as sequences of multidimensional arrays\ncalled tensors. How can hidden, evolving trends in such data be extracted while\npreserving the tensor structure? The model that is traditionally used is the linear\ndynamical system (LDS) with Gaussian noise, which treats the latent state and\nobservation at each time slice as a vector. We present the multilinear dynamical\nsystem (MLDS) for modeling tensor time series and an expectation?maximization\n(EM) algorithm to estimate the parameters. The MLDS models each tensor observation in the time series as the multilinear projection of the corresponding member\nof a sequence of latent tensors. The latent tensors are again evolving with respect\nto a multilinear projection. Compared to the LDS wi

In [178]:
idx = 4550
keywords = get_keywords(idx,docs)
print_keywords(idx,keywords,docs)


--------Title-------
Multilinear Dynamical Systems for Tensor Time Series

--------Abstract-------
Many scientific data occur as sequences of multidimensional arrays called tensors.  How can hidden, evolving trends in such data be extracted while preserving the tensor structure?  The model that is traditionally used is the linear dynamical system (LDS), which treats the observation at each time slice as a vector.  In this paper, we propose the multilinear dynamical system (MLDS) for modeling tensor time series and an expectation-maximization (EM) algorithm to estimate the parameters.  The MLDS models each time slice of the tensor time series as the multilinear projection of a corresponding member of a sequence of latent, low-dimensional tensors.  Compared to the LDS with an equal number of parameters, the MLDS achieves higher prediction accuracy and marginal likelihood for both simulated and real datasets.

--------Keywords-------
tensor 0.819
vec 0.393
time seri 0.121
model 0.112
ser

In [180]:
#creating the pickle files
import pickle
pickle.dump(count_vector,open('count_vector.pkl','wb'))
pickle.dump(feature_names,open('feature_names.pkl','wb'))
pickle.dump(tfidf_transformer,open('tfidf_transformer.pkl','wb'))